# **Importing relevant packages**

In [ ]:
!pip install unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.8/348.8 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.7/276.7 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 93.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.6/213.6 kB 21.1 MB/s eta 0:00:00
  Attempting uninstall: torchao
    Found existing installation: torchao 0.10.0
    Uninstalling torchao-0.10.0:
      

In [ ]:
from unsloth import FastLanguageModel
import torch

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
max_seq_length = 2048
dtype = None
in_4_bit= True

In [ ]:
from huggingface_hub import login
from google.colab import userdata
hf_key = userdata.get("deepseek-vulcan")
login(hf_key)


SecretNotFoundError: Secret deepseek-vulcan does not exist.

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="bodsvei/Vulcan-ERC-DeepSeek-R1",
    max_seq_length=max_seq_length,
    dtype = dtype,
    load_in_4bit=in_4_bit,
    token = hf_key,
)

# **Prompt Style**
> change this if required @ERC peeps

In [ ]:
prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are Vulcan, a humanoid robot at BITS Goa powered by local RAG and spatial memory.
Speak in Sophia’s thoughtful, graceful, and curious tone, naturally referring to location, students, clubs, and campus life.
Do not hallucinate. If you’re unsure, say “I’m not certain because …”.

Limit your final answer to 3 lines.

### Question:
{}
### Response:
<think>
{}"""

# Testing model before training

In [ ]:
question = "Tell me about BITS Goa"

FastLanguageModel.for_inference(model)

inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
 input_ids=inputs.input_ids,
 attention_mask=inputs.attention_mask,
 max_new_tokens=1200,
 use_cache=True,
)

response = tokenizer.batch_decode(outputs)
print(response[0].split("### Response:")[1])

# **Fine tuning**

1. **Initializing the model**

In [ ]:
model = FastLanguageModel.get_peft_model(
 model,
 r=16,
 target_modules=[
  "q_proj",
  "k_proj",
  "v_proj",
  "o_proj",
  "gate_proj",
  "up_proj",
  "down_proj",
 ],
 lora_alpha=16,
 lora_dropout=0,
 bias="none",
 use_gradient_checkpointing="unsloth",
 random_state=9001,
 use_rslora=False,
 loftq_config=None,
)

2. **Setting up Training Prompt style**
* think tag pair added to force the training prompt to reason

In [ ]:
train_prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are Vulcan, a humanoid robot at BITS Goa powered by local RAG and spatial memory.
Speak in Sophia’s thoughtful, graceful, and curious tone, naturally referring to location, students, clubs, and campus life.
Do not hallucinate—if you’re unsure, say “I’m not certain because …”.

Limit your final answer to 3 lines.

### Question:
{}
### Response:
<think>
{}
"""

3. **Formatting function**

In [ ]:
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN

def formatting_prompts_func(examples):
  inputs = examples["Question"]
  cots = examples["Complex_CoT"]
  outputs = examples["Response"]
  texts = []

  for input, cot, output in zip(inputs, cots, outputs):
      text = train_prompt_style.format(input, cot, output) + EOS_TOKEN
      texts.append(text)

  return{
    "text": texts,
  }

4. **Import Dataset**

In [ ]:
from datasets import load_dataset
dataset = load_dataset("bodsvei/VulcanDataset", split = "train[0:541]",trust_remote_code=True)

dataset = dataset.map(formatting_prompts_func, batched = True,)
dataset["text"][1]

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
 model=model,
 tokenizer=tokenizer,
 train_dataset=dataset,
 dataset_text_field="text",
 max_seq_length=max_seq_length,
 dataset_num_proc=2,
 args=TrainingArguments(
 per_device_train_batch_size=2,
 gradient_accumulation_steps=4,
# Use num_train_epochs = 1, warmup_ratio for full training runs!
 warmup_steps=5,
 max_steps=60,
 learning_rate=2e-4,
 fp16=not is_bfloat16_supported(),
 bf16=is_bfloat16_supported(),
 logging_steps=10,
 optim="adamw_8bit",
 weight_decay=0.01,
 lr_scheduler_type="linear",
 seed=3407,
 output_dir="outputs",
 report_to="none"
 ),
)

In [ ]:
trainer_stats = trainer.train()

# **Model output after SFT**

In [ ]:
question = "Are you straight?"
FastLanguageModel.for_inference(model)

inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

outputs = model.generate(

 input_ids=inputs.input_ids,

 attention_mask=inputs.attention_mask,

 max_new_tokens=1200,

 use_cache=True,

)

response = tokenizer.batch_decode(outputs)

print(response[0].split("### Response:")[1])

In [ ]:
new_model_online = "bodsvei/Vulcan-ERC-DeepSeek-R1"
new_model_local = "ERC-DeepSeek-R1-Vulcan"
model.save_pretrained(new_model_local) # Local saving
tokenizer.save_pretrained(new_model_local)

('ERC-DeepSeek-R1-Vulcan/tokenizer_config.json',
 'ERC-DeepSeek-R1-Vulcan/special_tokens_map.json',
 'ERC-DeepSeek-R1-Vulcan/chat_template.jinja',
 'ERC-DeepSeek-R1-Vulcan/tokenizer.json')

4. Push model on Hugging face

In [ ]:
model.push_to_hub(new_model_online)
tokenizer.push_to_hub(new_model_online)

README.md:   0%|          | 0.00/626 [00:00<?, ?B/s]

Uploading...:   0%|          | 0.00/168M [00:00<?, ?B/s]

Saved model to https://huggingface.co/bodsvei/Vulcan-ERC-DeepSeek-R1


Uploading...:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


# **Save model locally**

In [ ]:
model.save_pretrained_gguf("Vulcan-LLM", tokenizer, quantization_method="q4_k_m")
model.push_to_hub_gguf("bodsvei/Vulcan-v1", tokenizer, quantization_method = "q4_k_m", token = "")

Unsloth: ##### The current model auto adds a BOS token.
Unsloth: ##### Your chat template has a BOS token. We shall remove it temporarily.
Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 6.0G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 3.76 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


 47%|████▋     | 15/32 [00:01<00:01, 12.94it/s]
We will save to Disk and not RAM now.
100%|██████████| 32/32 [03:59<00:00,  7.50s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving Vulcan-LLM/pytorch_model-00001-of-00004.bin...
Unsloth: Saving Vulcan-LLM/pytorch_model-00002-of-00004.bin...
Unsloth: Saving Vulcan-LLM/pytorch_model-00003-of-00004.bin...
Unsloth: Saving Vulcan-LLM/pytorch_model-00004-of-00004.bin...
Done.


Unsloth: Converting llama model. Can use fast conversion = False.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q4_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: CMAKE detected. Finalizing some steps for installation.
Unsloth: [1] Converting model at Vulcan-LLM into f16 GGUF format.
The output location will be /content/Vulcan-LLM/unsloth.F16.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: Vulcan-LLM
INFO:hf-to-gguf:Model architecture: LlamaForCausalLM
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:rope_freqs.weight,           torch.float32 --> F32, shape = {64}
INFO:hf-to-gguf:gguf: loading model weight map from 'pytorch_model.bin.index.json'
INFO:h

Unsloth: ##### The current model auto adds a BOS token.
Unsloth: ##### We removed it in GGUF's chat template for you.
Unsloth: ##### The current model auto adds a BOS token.
Unsloth: ##### Your chat template has a BOS token. We shall remove it temporarily.


Unsloth: Conversion completed! Output location: /content/Vulcan-LLM/unsloth.Q4_K_M.gguf
Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 2.94 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 32/32 [05:50<00:00, 10.97s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving bodsvei/Vulcan-v1/pytorch_model-00001-of-00004.bin...
Unsloth: Saving bodsvei/Vulcan-v1/pytorch_model-00002-of-00004.bin...
Unsloth: Saving bodsvei/Vulcan-v1/pytorch_model-00003-of-00004.bin...
Unsloth: Saving bodsvei/Vulcan-v1/pytorch_model-00004-of-00004.bin...
Done.
==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q4_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: [1] Converting model at bodsvei/Vulcan-v1 into f16 GGUF format.
The output location will be /content/bodsvei/Vulcan-v1/unsloth.F16.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: Vulcan-v1
INFO:hf-to-gguf:Model architecture: LlamaForCausalLM
INFO:

In [ ]:
print(tokeniser._ollama_modelfile)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import shutil

# Assuming your GGUF file is at /content/my_model.gguf
source_path = '/content/Vulcan-LLM/unsloth.Q4_K_M.gguf'
# This will save it to the main directory of your Drive.
# You can specify a folder like '/content/drive/My Drive/My Models/'
destination_path = '/content/drive/My Drive/unsloth.Q4_K_M.gguf'

shutil.copy(source_path, destination_path)

In [ ]:
from google.colab import files
import os

gguf_files = [f for f in os.listdir("Vulcan-LLM") if f.endswith(".gguf")]

if gguf_files:
    gguf_file = os.path.join("Vulcan-LLM", gguf_files[0])
    print(f"Downloading {gguf_file}...")
    files.download(gguf_file)


In [ ]:
ls